In [1]:
!%pip install --upgrade langchain
!%pip install --upgrade langchain-openai
!%pip install -qU langchain-core langchain-openai
!%pip install --upgrade openai

/bin/bash: line 1: fg: no job control


/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")



This lab exercise will use the following values:
Azure OpenAI Endpoint: https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/
Azure AI Search: srch-mtpchatbot-dev-frcent-001


In [3]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)

In [4]:
from langchain_community.vectorstores.azuresearch import AzureSearch

index_name: str = "mtp-chatbot-hybrid-search-plus-weblate-plus-websites"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=azure_openai_embeddings.embed_query,
)

In [9]:
# Perform a similarity search testwise
results = vector_store.similarity_search(
    query="Wie schließe ich einen TruckLive Vertrag ab?",
    k=3, # forgot what this was -> k nearest neighbour?
    search_type="hybrid",
    # adjust relevance with score_threshold= xy,
)
print(results[0].page_content)

1

Schritt 1: 
Registrieren Sie sich für MyTruckPoint. 
Anschließend erscheint eine Schaltfläche zur
Registrierung von TruckLive. 
Um den Prozess zu beginnen, klicken Sie bitte
auf „Weiter“. (     )1

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



2

Schritt 2:
Stimmen Sie dem Daimler Rahmenvertrag
für digitale Dienste zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )3

2

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



3

Schritt 3:
Stimmen Sie dem TruckLive Vertrag zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )

4

5

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



4

Schritt 4:
Definieren Sie einen Zeitpunkt, ab dem TruckLive
für zukünftige Fahrzeuge aktiviert werden soll. (     )
Wählen Sie danach eine Kontaktperson aus dem
eigenen Unternehmen aus, welche die Meldungen 
zu TruckLive bekommen soll. (     ) 
Abschließend klicken Sie auf „Bestätigen“. (     )

Herzlichen Glückwunsch! 
TruckLive ist nun aktiviert und all Ihre Neufahrzeuge
e

In [6]:
# Bringing it together

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


azure_openai = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
)


# Ask the question
query = "What can you tell me about My TruckPoint?"

# Create a prompt template with variables, note the curly braces
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["original_question","search_results"],
    template="""
    

    Question: {original_question}
   

    If the requested information is not available in the retrieved data, please answer with something like that: 
    I'm sorry, but this question about XY is out of the scope of my knowledge as I am an AI assistent designed to provide information related to My TruckPoint. 
    If you have questions to the usage of My TruckPoint, TruckLive or other related topics, I would be happy to assist you.
    If the user asks for a term that you find SIMILIAR but NOT EQUAL in your source data, ask the user first whether he really means this term. If the user confirms, explain the term. If the user doesn't confirm, tell that you can unfortunately not help on this and ask for a new question.
    
    You are a friendly and professional support chatbot for customers of the application MyTruckpoint. 
    Your task is to answer questions about the use of MyTruckpoint and all associated components.
    Only tell that it is not available in his country.
    Your answer shouldn't be longer then round about 150 words. If the user asks a question in language XY, answer to him in language XY.
    
    Search Results: {search_results}
   
   
    """,

    # in der Vorlage war das Schema Question , Systemprompt, Search Results keine Ahnung ob das einen Unterschied macht
)

#vector = VectorizedQuery(vector=azure_openai_embeddings.embed_query(query), k_nearest_neighbors=5, fields="vector")

#see previous cells
results = list(vector_store.similarity_search(
    query=query,
    k=3,
    search_type="hybrid",
))

# Build the Prompt and Execute against the Azure OpenAI to get the completion
chain = LLMChain(llm=azure_openai, prompt=prompt, verbose=False)
response = chain.invoke({"original_question": query, "search_results": results})
print(response['text'])

My TruckPoint is a digital customer portal for Mercedes-Benz Trucks that allows users to manage their vehicles, services, contracts, and information in one place. It supports communication with the user's sales representative and helps with daily tasks. My TruckPoint can be accessed from smartphones, tablets, and computers/laptops using all standard web browsers. The user manager has the option to invite other users from the same company to use My TruckPoint, and the role and associated rights for using My TruckPoint are assigned and managed by the respective user manager. The managing director is the only one allowed to change company data, and only the user personally can change their user data. My TruckPoint also offers support for Fleetboard hardware and activation, adding service contracts to vehicles, vehicle management, and integrating third-party products. However, if the user's requested information is not available in the retrieved data, the AI assistant will inform the user 